In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import transforms
import torchvision.models as models

from sklearn.model_selection import train_test_split

from PIL import Image

In [2]:
torch.manual_seed(42)

In [3]:
df = pd.read_csv("/home/krish/Datasets/fashion_mnist/fashion-mnist_train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
custom_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
class customDataset(Dataset):

    def __init__(self, features, labels, transforms):
        super().__init__()
        self.features = features
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        image = self.features[index].reshape(28,28)
        image = image.astype(np.uint8)
        image = np.stack([image]*3, axis=-1)
        image = Image.fromarray(image)

        image = self.transforms(image)

        return image, torch.tensor(self.labels[index], dtype=torch.long)

In [8]:
train_dataset = customDataset(X_train, y_train, transforms=custom_transforms)
test_dataset = customDataset(X_test, y_test, transforms=custom_transforms)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [10]:
vgg16 = models.vgg16(pretrained=True)

/home/krish/miniconda3/envs/torchenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/krish/miniconda3/envs/torchenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
for param in vgg16.features.parameters():
    param.requires_grad = False

In [12]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

vgg16 = vgg16.to(device)

In [13]:
epochs = 50
learning_rate = 0.001

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [15]:
for epoch in range(epochs):

    Epoch_loss = 0
    for batch_features, batch_labels in train_loader:

        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        output = vgg16(batch_features)

        loss = criterion(output, batch_labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        Epoch_loss += loss.item()

    avg_epoch_loss = Epoch_loss/len(batch_features)
    print(f"Loss after Epoch {epoch+1}: {avg_epoch_loss}")

Loss after Epoch 1: 4.703159071272239
Loss after Epoch 2: 3.3773375515593216
Loss after Epoch 3: 2.9372054007835686
Loss after Epoch 4: 2.616177542018704
Loss after Epoch 5: 2.4316434050851967
Loss after Epoch 6: 2.2855767420842312
Loss after Epoch 7: 2.148963691521203
Loss after Epoch 8: 1.9309064302651677
Loss after Epoch 9: 1.8485559804539662
Loss after Epoch 10: 1.754582381390719
Loss after Epoch 11: 1.766329962942109
Loss after Epoch 12: 1.6165637952290126
Loss after Epoch 13: 1.4061846072145272
Loss after Epoch 14: 1.5012081569757356
Loss after Epoch 15: 1.512343488060651
Loss after Epoch 16: 1.4921564131072955
Loss after Epoch 17: 1.2491708460872815
Loss after Epoch 18: 1.3172402975014847
Loss after Epoch 19: 1.3053437740109075
Loss after Epoch 20: 1.222762259640831
Loss after Epoch 21: 1.2965321442852655
Loss after Epoch 22: 1.2155123878661698
Loss after Epoch 23: 1.1849590594447363
Loss after Epoch 24: 1.1069969962190953
Loss after Epoch 25: 1.1514047908949578
Loss after Epoch

In [16]:
# Evaluating the model
vgg16.eval()

# Evaluation Code on test data

total = 0
correct = 0

with torch.no_grad():

    for batch_features, batch_labels in test_loader:

        batch_features, batch_labels = batch_features.to(device=device), batch_labels.to(device=device)

        outputs = vgg16(batch_features)
        _, predicted = torch.max(outputs, 1)
        
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

    print(f"Test accuracy: {correct/total}")

Test accuracy: 0.9285


In [17]:
# Evaluation Code on test data

total_train = 0
correct_train = 0

with torch.no_grad():

    for batch_features, batch_labels in train_loader:

        batch_features, batch_labels = batch_features.to(device=device), batch_labels.to(device=device)

        outputs = vgg16(batch_features)
        _, predicted = torch.max(outputs, 1)
        
        total_train += batch_labels.shape[0]
        correct_train += (predicted == batch_labels).sum().item()

    print(f"Train accuracy: {correct_train/total_train}")

Train accuracy: 0.9993125
